In [17]:
import pandas as pd
import apikeys
import requests
import googleapiclient
from google.cloud import vision
from google.cloud.vision import types
import io
from sklearn.feature_extraction.text import CountVectorizer


In [73]:
image_df = pd.read_csv('image_data.csv')
image_df = image_df.dropna(axis=0, how='any' )
image_df.head()

,asin,overall,imUrl
0,0001473727,5.000000,http://ecx.images-amazon.com/images/I/512M299K...
1,000100039X,4.666667,http://ecx.images-amazon.com/images/I/81ZKLPiv...
2,0001473905,5.000000,http://ecx.images-amazon.com/images/I/61KPC59B...
3,0001473123,5.000000,http://ecx.images-amazon.com/images/I/317G8R2T...
4,0002005395,4.000000,http://ecx.images-amazon.com/images/I/415HPT70...


In [82]:
def detect_labels_uri(uri, dataframe):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    client = vision.ImageAnnotatorClient()
    image = types.Image()
    image.source.image_uri = uri
    text = []
    response = client.label_detection(image=image)
    labels = response.label_annotations
    for label in labels:
        text.append(label.description)
    dataframe['text']= [text]
    dataframe['labels_qty'] = len(labels)

In [83]:
detect_labels_uri('http://ecx.images-amazon.com/images/I/5122XJRJyrL.jpg', dummy_df)
dummy_df

,text,labels_qty
0,"[text, font, straw]",3


In [94]:
small_df = image_df.head(1000).copy()
combined_df = ([])

for index, row in small_df.iterrows():
    try:
        url = row['imUrl']
        df_row = row
        detect_labels_uri(url, df_row)
        combined_df.append(df_row)
    except:
        pass

In [102]:
combined_df = pd.DataFrame(combined_df)
combined_df.to_csv('combined.csv', index=False)

In [103]:

good_df = combined_df.loc[image_df['overall'] >4.0]
good_df.to_csv('good.csv', index=False)

In [104]:
    
bad_df = combined_df.loc[image_df['overall']<2]
bad_df.to_csv('bad.csv', index=False)